Name: Trần Văn Lực
MSSV : 20521587

# **K-NEAREST NEIGHBOR**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Setting up environment

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import scipy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
%cd '/content/drive/MyDrive/Covid 19 Risk Prediction/Luc_KNN'
%ls

/content/drive/.shortcut-targets-by-id/1VG1Uqoa-0jVp0m7WDEmIU2dw_FP-wLtl/Covid 19 Risk Prediction/Luc_KNN
knn_model_GR_smdata_v1  knn_model_GR_v1  knn_model_v1  knn_model_v3
knn_model_GR_smdata_v2  knn_model_GR_v2  knn_model_v2  Luc_KNN.ipynb


In [ ]:
data_train = pd.read_csv('../lan1/train.csv')
data_test = pd.read_csv('../lan1/test.csv')

## Model


### Parameter Tuning


Parameter Tuning:

        1. n_neighbors : Number of neighbors to use by default for kneighbors queries.
        2. algorithm: Algorithm used to compute the nearest neighbors.
        3. metric: Metric to use for distance computation.

* Để tìm được n_neighbors tối ưu nhằm tạo ra một bộ phân loại KNN mạnh mẽ trên bộ dữ liệu lớn như vậy thì phải thực hiện tìm phạm vi giá trị cho n_neighbors tối ưu trên một phần của bộ dữ liệu lớn trước (Khi thực hiện tuning trên một phần của bộ dữ liệu với GridSearchCV thì khoảng tuning của n_neighbors sẽ được thực hiện trên phạm vi rộng từ đó tìm ra phạm vi giá trị n_neighbors tối ưu để thực hiện train tunning trên toàn bộ dữ liệu )

### Tuning n_neighbors

#### Tuning with small train set v1

In [ ]:
best_params_list = []
# Instantiate model 
knn  = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(5,30),
              'algorithm': ['kd_tree'],
              'metric':['euclidean','manhattan','minkowski']} 

# Tunning parameters with GridSearchCV
knn_gscv = GridSearchCV(knn,param_grid,cv=5,scoring='accuracy')

# Load data , train the model
# v1 train in stunning in small train set
# undersampling the train set
data_train = pd.read_csv('../lan1/train.csv')
atrisk = data_train[data_train.AT_RISK==1][:100000]
nonrisk = data_train[data_train.AT_RISK==0][:100000]
part_train = pd.concat([nonrisk, atrisk])
train_y = part_train.AT_RISK.to_numpy()
train_X = part_train.drop(columns = ['AT_RISK']).to_numpy()

# fitting the model for grid search
start = time.time()
knn_gscv.fit(train_X,train_y)
end = time.time()
print(f"Time to train for knn v1: {end - start:.2f} s")

# Best parameters
print(f"Best parameters of knn v1: {knn_gscv.best_params_}")
best_params_list.append(knn_gscv.best_params_)

# Accuracy for our training dataset
accuracy = knn_gscv.best_score_
print(f"Accuracy for training dataset lan with tuning is : {accuracy:.2f}")

# save
knnPickle = open('knn_model_GR_smdata_v1', 'wb') 
pickle.dump(knn_gscv.best_estimator_, knnPickle)

Time to train for knn v1: 1285.24 s
Best parameters of knn v1: {'algorithm': 'kd_tree', 'metric': 'manhattan', 'n_neighbors': 9}
Accuracy for training dataset lan 3 with tuning is : 0.86


In [ ]:
# Load data
data_test = pd.read_csv('../lan1/test.csv')
test_y = data_test.AT_RISK.to_numpy()
test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()

# Load model to predict
knn = pickle.load(open('knn_model_GR_smdata_v1', 'rb'))
pred = knn.predict(test_x) 
knnPickle.close()
print('KNeighborsClassifier \n', classification_report(test_y, pred))
print(f"Accuracy for test dataset: {accuracy_score(test_y, pred):.2f}")

KNeighborsClassifier 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95    274237
           1       0.60      0.60      0.60     32357

    accuracy                           0.91    306594
   macro avg       0.77      0.77      0.77    306594
weighted avg       0.91      0.91      0.91    306594

Accuracy for test dataset: 0.91


* n_neighbors từ quá trình train trên cho thấy n_neighbors tối ưu = 9 
* Tiếp theo sẽ train trên train set lớn hơn nhưng với khoản n_neighbors nhỏ hơn gần 9 đã tìm ra ở trên trên

#### Tunning with small train set v2


In [ ]:
best_params_list = []
# Instantiate model 
knn  = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(5,15),
              'algorithm': ['kd_tree'],
              'metric':['euclidean','manhattan','minkowski']} 

# Tunning parameters with GridSearchCV
knn_gscv = GridSearchCV(knn,param_grid,cv=5,scoring='accuracy')

# Load data , train the model
# v2 train in stunning in small train set
# undersampling the train set
data_train = pd.read_csv('../lan1/train.csv')
atrisk = data_train[data_train.AT_RISK==1][:200000]
nonrisk = data_train[data_train.AT_RISK==0][:200000]
part_train = pd.concat([nonrisk, atrisk])
train_y = part_train.AT_RISK.to_numpy()
train_X = part_train.drop(columns = ['AT_RISK']).to_numpy()

# fitting the model for grid search
start = time.time()
knn_gscv.fit(train_X,train_y)
end = time.time()
print(f"Time to train for knn v2: {end - start:.2f} s")

# Best parameters
print(f"Best parameters of knn v2: {knn_gscv.best_params_}")
best_params_list.append(knn_gscv.best_params_)

# Accuracy for our training dataset
accuracy = knn_gscv.best_score_
print(f"Accuracy for small train set with tuning is : {accuracy:.2f}")

# save
knnPickle = open('knn_model_GR_smdata_v2', 'wb') 
pickle.dump(knn_gscv.best_estimator_, knnPickle)

Time to train for knn v2: 2421.91 s
Best parameters of knn v2: {'algorithm': 'kd_tree', 'metric': 'manhattan', 'n_neighbors': 13}
Accuracy for small train set with tuning is : 0.86


In [ ]:
# Load data
data_test = pd.read_csv('../lan1/test.csv')
test_y = data_test.AT_RISK.to_numpy()
test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()

# Load model to predict
knn = pickle.load(open('knn_model_GR_smdata_v2', 'rb'))
pred = knn.predict(test_x) 
knnPickle.close()
print('KNeighborsClassifier \n', classification_report(test_y, pred))
print(f"Accuracy for test dataset: {accuracy_score(test_y, pred):.2f}")

KNeighborsClassifier 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95    274237
           1       0.60      0.60      0.60     32357

    accuracy                           0.91    306594
   macro avg       0.77      0.77      0.77    306594
weighted avg       0.91      0.91      0.91    306594

Accuracy for test dataset: 0.91


* n_neighbors từ quá trình train trên cho thấy n_neighbors tối ưu = 13

* **Qua 2 lần tunning với bộ dữ liệu nhỏ thì có thể đưa ra phạm vi giá trị n_neighbors tối ưu là từ 7 đến 14**

#### Tuning all dataset

In [ ]:
best_params_list = []
# Instantiate model 
knn  = KNeighborsClassifier()
param_grid = {'n_neighbors': [8,9,10,11,12,13,14],
              'algorithm': ['kd_tree'],} 

# Tunning parameters with GridSearchCV
knn_gscv = GridSearchCV(knn,param_grid,cv=5,scoring='accuracy')

# Load data , train the model
data_train = pd.read_csv('../lan1/train.csv')
train_y = data_train.AT_RISK.to_numpy()
train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()

# fitting the model for grid search
start = time.time()
knn_gscv.fit(train_X,train_y)
end = time.time()
print(f"Time to train : {end - start:.2f} s")

# Best parameters
print(f"Best n_neighbors: {knn_gscv.best_params_}")
best_params_list.append(knn_gscv.best_params_)

# Accuracy for training dataset
accuracy = knn_gscv.best_score_
print(f"Accuracy for train set : {accuracy:.2f}")

# save
knnPickle = open('knn_model_GR_v1', 'wb') 
pickle.dump(knn_gscv.best_estimator_, knnPickle)

Time to train : 1961.99 s
Best n_neighbors: {'algorithm': 'kd_tree', 'n_neighbors': 13}
Accuracy for train set : 0.92


In [ ]:
# Load data
data_test = pd.read_csv('../lan1/test.csv')
test_y = data_test.AT_RISK.to_numpy()
test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()

# Load model to predict
knn = pickle.load(open('knn_model_GR_v1', 'rb'))
pred = knn.predict(test_x) 
knnPickle.close()
print('KNeighborsClassifier \n', classification_report(test_y, pred))
print(f"Accuracy for test dataset: {accuracy_score(test_y, pred):.2f}")

KNeighborsClassifier 
               precision    recall  f1-score   support

           0       0.94      0.97      0.96    274237
           1       0.66      0.51      0.58     32357

    accuracy                           0.92    306594
   macro avg       0.80      0.74      0.77    306594
weighted avg       0.91      0.92      0.92    306594

Accuracy for test dataset: 0.92


 Qua các lần tuning trên thì giá trị của n_neighbors tốt nhất để tạo ra một bộ phân loại KNN mạnh mẽ trên tập dữ liệu Covid 19 Risk Prediction là 9 hoặc 13

### Tuning metric 

In [ ]:
best_params_list = []
# Instantiate model 
knn  = KNeighborsClassifier()
param_grid = {'n_neighbors': [9,13],
              'algorithm': ['kd_tree'],
              'metric':['euclidean','manhattan','minkowski']} 

# Tunning parameters with GridSearchCV
knn_gscv = GridSearchCV(knn,param_grid,cv=5,scoring='accuracy')

# Load data , train the model
data_train = pd.read_csv('../lan1/train.csv')
train_y = data_train.AT_RISK.to_numpy()
train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()

# fitting the model for grid search
start = time.time()
knn_gscv.fit(train_X,train_y)
end = time.time()
print(f"Time to train : {end - start:.2f} s")

# Best parameters
print(f"Best metric : {knn_gscv.best_params_}")
best_params_list.append(knn_gscv.best_params_)

# Accuracy for training dataset
accuracy = knn_gscv.best_score_
print(f"Accuracy for train set : {accuracy:.2f}")

# save
knnPickle = open('knn_model_GR_v2', 'wb') 
pickle.dump(knn_gscv.best_estimator_, knnPickle)

Time to train : 1723.23 s
Best metric : {'algorithm': 'kd_tree', 'metric': 'manhattan', 'n_neighbors': 9}
Accuracy for train set : 0.92


In [ ]:
# Load data
data_test = pd.read_csv('../lan1/test.csv')
test_y = data_test.AT_RISK.to_numpy()
test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()

# Load model to predict
knn = pickle.load(open('knn_model_GR_v2', 'rb'))
pred = knn.predict(test_x) 
knnPickle.close()
print('KNeighborsClassifier \n', classification_report(test_y, pred))
print(f"Accuracy for test dataset: {accuracy_score(test_y, pred):.2f}")

KNeighborsClassifier 
               precision    recall  f1-score   support

           0       0.95      0.97      0.96    274237
           1       0.65      0.53      0.58     32357

    accuracy                           0.92    306594
   macro avg       0.80      0.75      0.77    306594
weighted avg       0.91      0.92      0.92    306594

Accuracy for test dataset: 0.92


Qua các lần tuning với bộ dữ liệu và tham số khác nhưng các model cho kết quả tốt nhất đều sử dụng **metric** là **manhattan**  ->>  **manhattan** được coi là metric phù hợp nhất để tạo nên một bộ phân loại **KNN mạnh mẽ** trên bộ dữ liệu **Covid 19 Risk Prediction**

### Experiment

In [6]:
time_list = []
acc_list = []
f1_list = []
pre_list = []
rec_list = []

In [11]:
# Instantiate model 
knn  = KNeighborsClassifier(n_neighbors = 5,algorithm ='kd_tree',metric='manhattan')
# Load data , train the model
for i in range(1,4):
  data_train = pd.read_csv('../lan'+str(i)+'/train.csv')
  train_y = data_train.AT_RISK.to_numpy()
  train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()
  data_test = pd.read_csv('../lan'+str(i)+'/test.csv')
  test_y = data_test.AT_RISK.to_numpy()
  test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()
  # fitting the model 
  start = time.time()
  knn.fit(train_X,train_y)
  end = time.time()
  print(f"Time to train for knn lan{i}: {end - start:.4f} s")
  time_list.append(end - start);
  # predict
  pred = knn.predict(test_x) 
  acc_list.append(accuracy_score(test_y, pred))
  f1_list.append(f1_score(test_y, pred))
  pre_list.append(precision_score(test_y, pred))
  rec_list.append(recall_score(test_y, pred))

print(f"n_neighbors = 5")
print(f"Train time: {(sum(time_list) / len(time_list)):.2f}")
print(f"Accuracy: {(sum(acc_list) / len(acc_list))*100:.2f}")
print(f"F1_score: {(sum(f1_list) / len(f1_list))*100:.2f}")
print(f"Precision_score: {(sum(pre_list) / len(pre_list))*100:.2f}")
print(f"Recall_score: {(sum(rec_list) / len(rec_list))*100:.2f}")

Time to train for knn lan1: 4.0708 s
Time to train for knn lan2: 3.7731 s
Time to train for knn lan3: 3.9849 s
n_neighbors = 5
Train time: 4.03
Accuracy: 92.00
F1_score: 58.11
Precision_score: 65.14
Recall_score: 52.46


In [9]:
mt = ['euclidean','manhattan','minkowski']
for k in mt:
  # Instantiate model 
  knn  = KNeighborsClassifier(n_neighbors = 9,algorithm ='kd_tree',metric = k)
  # Load data , train the model
  for i in range(1,4):
    data_train = pd.read_csv('../lan'+str(i)+'/train.csv')
    train_y = data_train.AT_RISK.to_numpy()
    train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()
    data_test = pd.read_csv('../lan'+str(i)+'/test.csv')
    test_y = data_test.AT_RISK.to_numpy()
    test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()
    # fitting the model 
    start = time.time()
    knn.fit(train_X,train_y)
    end = time.time()
    time_list.append(end - start);
    # predict
    pred = knn.predict(test_x) 
    acc_list.append(accuracy_score(test_y, pred))
    f1_list.append(f1_score(test_y, pred))
    pre_list.append(precision_score(test_y, pred))
    rec_list.append(recall_score(test_y, pred))

  print(f"n_neighbors = 9,metric={k}")
  print(f"Train time: {(sum(time_list) / len(time_list)):.2f}")
  print(f"Accuracy: {(sum(acc_list) / len(acc_list))*100:.2f}")
  print(f"F1_score: {(sum(f1_list) / len(f1_list))*100:.2f}")
  print(f"Precision_score: {(sum(pre_list) / len(pre_list))*100:.2f}")
  print(f"Recall_score: {(sum(rec_list) / len(rec_list))*100:.2f}")

n_neighbors = 9,metric=euclidean
Train time: 4.21
Accuracy: 92.02
F1_score: 58.06
Precision_score: 65.41
Recall_score: 52.20
n_neighbors = 9,metric=manhattan
Train time: 4.18
Accuracy: 92.02
F1_score: 58.07
Precision_score: 65.38
Recall_score: 52.24
n_neighbors = 9,metric=minkowski
Train time: 4.16
Accuracy: 92.02
F1_score: 58.08
Precision_score: 65.36
Recall_score: 52.27


In [7]:
# Instantiate model 
knn  = KNeighborsClassifier(n_neighbors = 13,algorithm ='kd_tree',metric='manhattan')
# Load data , train the model
for i in range(1,4):
  data_train = pd.read_csv('../lan'+str(i)+'/train.csv')
  train_y = data_train.AT_RISK.to_numpy()
  train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()
  data_test = pd.read_csv('../lan'+str(i)+'/test.csv')
  test_y = data_test.AT_RISK.to_numpy()
  test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()
  # fitting the model 
  start = time.time()
  knn.fit(train_X,train_y)
  end = time.time()
  print(f"Time to train for knn lan{i}: {end - start:.4f} s")
  time_list.append(end - start);
  # predict
  pred = knn.predict(test_x) 
  acc_list.append(accuracy_score(test_y, pred))
  f1_list.append(f1_score(test_y, pred))
  pre_list.append(precision_score(test_y, pred))
  rec_list.append(recall_score(test_y, pred))

print(f"n_neighbors = 13,algorithm ='kd_tree'")
print(f"Train time: {(sum(time_list) / len(time_list)):.2f}")
print(f"Accuracy: {(sum(acc_list) / len(acc_list))*100:.2f}")
print(f"F1_score: {(sum(f1_list) / len(f1_list))*100:.2f}")
print(f"Precision_score: {(sum(pre_list) / len(pre_list))*100:.2f}")
print(f"Recall_score: {(sum(rec_list) / len(rec_list))*100:.2f}")

Time to train for knn lan1: 4.6930 s
Time to train for knn lan2: 4.6336 s
Time to train for knn lan3: 4.5357 s
n_neighbors = 13,algorithm ='kd_tree'
Train time: 4.62
Accuracy: 92.06
F1_score: 57.73
Precision_score: 66.15
Recall_score: 51.22


In [15]:
# Instantiate model 
knn  = KNeighborsClassifier(n_neighbors = 13,algorithm ='ball_tree',metric='manhattan')
# Load data , train the model
for i in range(1,4):
  data_train = pd.read_csv('../lan'+str(i)+'/train.csv')
  train_y = data_train.AT_RISK.to_numpy()
  train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()
  data_test = pd.read_csv('../lan'+str(i)+'/test.csv')
  test_y = data_test.AT_RISK.to_numpy()
  test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()
  # fitting the model 
  start = time.time()
  knn.fit(train_X,train_y)
  end = time.time()
  print(f"Time to train for knn lan{i}: {end - start:.4f} s")
  time_list.append(end - start);
  # predict
  pred = knn.predict(test_x) 
  acc_list.append(accuracy_score(test_y, pred))
  f1_list.append(f1_score(test_y, pred))
  pre_list.append(precision_score(test_y, pred))
  rec_list.append(recall_score(test_y, pred))

print(f"n_neighbors = 13,algorithm ='ball_tree'")
print(f"Train time: {(sum(time_list) / len(time_list)):.2f}")
print(f"Accuracy: {(sum(acc_list) / len(acc_list))*100:.2f}")
print(f"F1_score: {(sum(f1_list) / len(f1_list))*100:.2f}")
print(f"Precision_score: {(sum(pre_list) / len(pre_list))*100:.2f}")
print(f"Recall_score: {(sum(rec_list) / len(rec_list))*100:.2f}")

Time to train for knn lan1: 2.5775 s
Time to train for knn lan2: 2.6622 s
Time to train for knn lan3: 2.7015 s
n_neighbors = 13,algorithm ='ball_tree'
Train time: 3.78
Accuracy: 91.98
F1_score: 57.07
Precision_score: 65.84
Recall_score: 50.52


In [12]:
# Instantiate model 
knn  = KNeighborsClassifier(n_neighbors = 30,algorithm ='kd_tree',metric='manhattan')
# Load data , train the model
for i in range(1,4):
  data_train = pd.read_csv('../lan'+str(i)+'/train.csv')
  train_y = data_train.AT_RISK.to_numpy()
  train_X = data_train.drop(columns = ['AT_RISK']).to_numpy()
  data_test = pd.read_csv('../lan'+str(i)+'/test.csv')
  test_y = data_test.AT_RISK.to_numpy()
  test_x = data_test.drop(columns = ['AT_RISK']).to_numpy()
  # fitting the model 
  start = time.time()
  knn.fit(train_X,train_y)
  end = time.time()
  print(f"Time to train for knn lan{i}: {end - start:.4f} s")
  time_list.append(end - start);
  # predict
  pred = knn.predict(test_x) 
  acc_list.append(accuracy_score(test_y, pred))
  f1_list.append(f1_score(test_y, pred))
  pre_list.append(precision_score(test_y, pred))
  rec_list.append(recall_score(test_y, pred))

print(f"n_neighbors = 30")
print(f"Train time: {(sum(time_list) / len(time_list)):.2f}")
print(f"Accuracy: {(sum(acc_list) / len(acc_list))*100:.2f}")
print(f"F1_score: {(sum(f1_list) / len(f1_list))*100:.2f}")
print(f"Precision_score: {(sum(pre_list) / len(pre_list))*100:.2f}")
print(f"Recall_score: {(sum(rec_list) / len(rec_list))*100:.2f}")

Time to train for knn lan1: 3.8353 s
Time to train for knn lan2: 3.6658 s
Time to train for knn lan3: 4.0797 s
n_neighbors = 30
Train time: 4.00
Accuracy: 91.99
F1_score: 57.53
Precision_score: 65.59
Recall_score: 51.34


**Bộ tham số tối ưu cho cho KNN trên bộ dữ liệu Covid 19 Risk Prediction:**

*   n_neighbors : 13
*   algorithm : kd_tree
*   metric : manhattan

